# **Computational Theory Assessment G00417529**

# **Imports**

In [1405]:
# numpy and unittest imports
import numpy as np
import unittest
# 32-bit unsigned integer alies to reduce verbosity of notebook
u32 = np.uint32
# Decorator for decorators, keeps function metadata intact
from functools import wraps
# Used for benchmarking function performance
import timeit


## **Introduction**
This section defines technical terms used for all the solutions in this notebook

### **Bitwise Terms and Operations**
**Bitwise Operations** are actions directly applied to individual bits of binary numbers. <br>
**Binary Words** are the natural unit of data a computer CPU can process with fixed lengths like 32bits (4 bytes) or 64 bits (8 bytes) <br><br>

**Bitwise AND ($\land$)/(`&`)**<br>
Compares each bit of two **binary words**.

For every bit position:
- The result bit is `1` only if both corresponding bits in the operands are `1`.
- Otherwise, the result bit is `0`.

<table style="border: 1px solid black; border-collapse: collapse; text-align: center; padding: 8px;">
  <tr>
    <th style="border: 1px solid black;">X</th>
    <th style="border: 1px solid black;">Y</th>
    <th style="border: 1px solid black;">X & Y</th>
  </tr>
  <tr>
    <td style="border: 1px solid black;">0</td>
    <td style="border: 1px solid black;">0</td>
    <td style="border: 1px solid black;">0</td>
  </tr>
  <tr>
    <td style="border: 1px solid black;">0</td>
    <td style="border: 1px solid black;">1</td>
    <td style="border: 1px solid black;">0</td>
  </tr>
  <tr>
    <td style="border: 1px solid black;">1</td>
    <td style="border: 1px solid black;">1</td>
    <td style="border: 1px solid black;">1</td>
  </tr>
</table><br>

**Bitwise OR (`|`)**<br>
For every bit position:
- The bit result is `1` if either bit is `1`.
- Otherwise, the result bit is `0`.

<table style="border: 1px solid black; border-collapse: collapse; text-align: center; padding: 8px;">
  <tr>
    <th style="border: 1px solid black;">X</th>
    <th style="border: 1px solid black;">Y</th>
    <th style="border: 1px solid black;">X | Y</th>
  </tr>
  <tr>
    <td style="border: 1px solid black;">0</td>
    <td style="border: 1px solid black;">0</td>
    <td style="border: 1px solid black;">0</td>
  </tr>
  <tr>
    <td style="border: 1px solid black;">0</td>
    <td style="border: 1px solid black;">1</td>
    <td style="border: 1px solid black;">1</td>
  </tr>
  <tr>
    <td style="border: 1px solid black;">1</td>
    <td style="border: 1px solid black;">1</td>
    <td style="border: 1px solid black;">1</td>
  </tr>
</table><br>

**Bitwise XOR ($\oplus$)/(`^`)**<br>
For every bit position:
- The bit result is `1` if both bits are different.
- Otherwise, the result bit is `0`.

<table style="border: 1px solid black; border-collapse: collapse; text-align: center; padding: 8px;">
  <tr>
    <th style="border: 1px solid black;">X</th>
    <th style="border: 1px solid black;">Y</th>
    <th style="border: 1px solid black;">X ^ Y</th>
  </tr>
  <tr>
    <td style="border: 1px solid black;">0</td>
    <td style="border: 1px solid black;">0</td>
    <td style="border: 1px solid black;">0</td>
  </tr>
  <tr>
    <td style="border: 1px solid black;">0</td>
    <td style="border: 1px solid black;">1</td>
    <td style="border: 1px solid black;">1</td>
  </tr>
  <tr>
    <td style="border: 1px solid black;">1</td>
    <td style="border: 1px solid black;">1</td>
    <td style="border: 1px solid black;">0</td>
  </tr>
</table><br>

**Bitwise NOT (`~`)**<br>
Flips every bit (`0→1`, `1→0`)

<table style="border: 1px solid black; border-collapse: collapse; text-align: center; padding: 8px;">
  <tr>
    <th style="border: 1px solid black;">X</th>
    <th style="border: 1px solid black;">~X</th>
  </tr>
  <tr>
    <td style="border: 1px solid black;">0</td>
    <td style="border: 1px solid black;">1</td>
  </tr>
  <tr>
    <td style="border: 1px solid black;">1</td>
    <td style="border: 1px solid black;">0</td>
  </tr>
</table><br>

## **Helper Functions**
Used to avoid duplicate code throughout solutions

### Local Unit Test Runner
Prevents variable redeclaration when running a specific test suite

In [1406]:
def run_local_scope_tests(test_class, verbosity_lvl=0):
    """
    Runs specific unittest.TestCase without redeclaring variables in the global scope.

    Parameters:
        test_class : unittest.TestCase
            The test class containing test methods.
        verbosity_lvl : int, optional
            Verbosity level for the test runner (default is 0).

    Returns:
        result : unittest.result.TestResult
            The result object containing information about the tests run.
    """

    # Load all test methods from the given TestCase class
    suite = unittest.TestLoader().loadTestsFromTestCase(test_class)

    # Create a test runner with the specified verbosity
    runner = unittest.TextTestRunner(verbosity=verbosity_lvl)

    # Run the test suite and return the result
    return runner.run(suite)

### Conversion Decorator
Automatically converts all plain int parameters of a given function into np.uint32 values, eliminating the need to implement individual type-checking logic for each method.

In [1407]:
# Constants for 32-bit unsigned integer range
U32_MAX = 2**32 - 1  # Maximum value that can be represented by an unsigned 32-bit integer
U32_MIN = 0          # Minimum value for an unsigned 32-bit integer (always 0)

def convert_to_u32(value: int) -> np.uint32:
    """
    Convert an integer to np.uint32, ensuring it fits within the valid 32-bit unsigned range.

    Parameters:
        value : int
            The integer value to be converted.

    Returns:
        np.uint32
            The converted 32-bit unsigned integer.

    Raises:
        ValueError
            If the input integer is outside the (0 to 2**32 - 1) range.
    """

    # Check that the value is within the allowed 32-bit unsigned integer range.
    # If not, raise an error to prevent silent overflow or data corruption.
    if (U32_MIN > value or value > U32_MAX):
        raise ValueError(f"Value {value} out of range for np.uint32.")

    # Safe to cast: convert Python int to NumPy's 32-bit unsigned integer type.
    return np.uint32(value)


In [ ]:
def force_u32(func):
    """
    Decorator that automatically converts all integer arguments passed to a function
    into np.uint32 to reduce verbosity and improve convenience.

    Purpose:
        - Simplifies code by automatically handling type conversion.
        - Prevents type errors by ensuring all integer inputs fit within 32-bit unsigned range.
        - Reduces repetitive code: validation is done once, here.

    Usage:
        Apply @force_u32 to functions that would normally take np.unit32 parameters and are called sapringly.
        This should not be used for performance-critical internal functions.
        When type is garanteed, don't use this decorator.
    """

    # Preserve metadata of the original function (name, docstring, etc.)
    # This ensures tools like `help()`, stack traces, and introspection reflect the original function.
    @wraps(func)
    def wrapper(*args, **kwargs):
        """
        Wrapper function that intercepts all arguments passed to the decorated function,
        converts plain integers to np.uint32 (with bounds checking), and then calls the original.
        """

        # --------- Convert positional arguments ---------
        # Iterate through all positional arguments (collected in the `args` tuple)
        # If an argument is a plain Python int, convert it to np.uint32 using the validated converter.
        # Non-int arguments (e.g., np.uint32, floats, strings) are passed through unchanged.
        new_args = tuple(
            convert_to_u32(a) if isinstance(a, int) else a
            for a in args
        )

        # --------- Convert keyword arguments ---------
        # Similarly process keyword arguments (stored in the `kwargs` dict).
        # Each value is checked and converted if it’s an int.
        new_kwargs = {
            key: convert_to_u32(value) if isinstance(value, int) else value
            for key, value in kwargs.items()
        }

        # --------- Call the original function ---------
        # After all conversions are done, call the original function `func`
        # using the safely converted arguments.
        return func(*new_args, **new_kwargs)

    # Return the wrapper so it replaces the original function definition.
    return wrapper


### Decorator (force_u32) Tests
#### Internal, fast functions (undecorated)

In [1409]:
# --- Internal, fast functions (undecorated) ---
def add(x: u32, y: u32) -> u32:
    """Add two np.uint32 numbers."""
    return x + y

def multiply(x: u32, y: u32) -> u32:
    """Multiply two np.uint32 numbers."""
    return x * y

#### User-facing, safe but slower functions (decorated)

In [1410]:
# --- User-facing, safe but slower functions (decorated) ---
@force_u32
def add_user_facing(x: int, y: int) -> u32:
    """User-facing add function, converts int inputs to np.uint32 automatically."""
    return add(x, y)

@force_u32
def multiply_user_facing(x: int, y: int) -> u32:
    """User-facing multiply function, converts int inputs to np.uint32 automatically."""
    return multiply(x, y)

#### Decorator Unit Tests

In [1411]:

class TestForceU32(unittest.TestCase):

    def test_alising(self):
        """Test that u32 alias works correctly"""
        self.assertEqual(u32(5), np.uint32(5))
        self.assertIsInstance(u32(5), np.uint32)
    
    def test_add_user_facing(self):
        """Test positional args for add_user_facing"""
        result = add_user_facing(7, 3)
        self.assertEqual(result, 10)
        self.assertIsInstance(result, np.uint32)
    
    def test_multiply_user_facing(self):
        """Test positional args for multiply_user_facing"""
        result = multiply_user_facing(3, 7)
        self.assertEqual(result, 21)
        self.assertIsInstance(result, np.uint32)

    def test_keyword_args(self):
        """Test that keyword arguments are converted"""
        result = add_user_facing(x=66, y=11)
        self.assertEqual(result, 77)
        self.assertIsInstance(result, np.uint32)

    def test_mixed_args(self):
        """Test combination of positional and keyword arguments"""
        result = multiply_user_facing(10, y=3)
        self.assertEqual(result, 30)
        self.assertIsInstance(result, np.uint32)

# Run local scope tests
run_local_scope_tests(TestForceU32)

----------------------------------------------------------------------
Ran 5 tests in 0.000s

OK


<unittest.runner.TextTestResult run=5 errors=0 failures=0>

### Preformance Demo

In [ ]:
def performance_demo():
    """Demonstrate performance difference between decorated and undecorated functions."""
    N = 10**7 # Number of iterations for timing (10^7)
    x, y = 10, 20 # Sample integer inputs

    # Undecorated internal function
    t_internal = timeit.timeit(lambda: add(u32(x), u32(y)), number=N)

    # Decorated user-facing function (with conversion)
    t_user = timeit.timeit(lambda: add_user_facing(x, y), number=N)

    print(f"Internal undecorated function time: {t_internal:.4f} s")
    print(f"User-facing decorated function time: {t_user:.4f} s")

    # Decorator adds overhead for type conversion (x3 slower), but provides safety and readability for user inputs.
    # This trade-off is often worthwhile in user-facing code, we prioritise safety and usability over raw performance.
    # but for inner loops, or methods where type safety is guaranteed, using undecorated functions is preferred.

# Run performance demonstration
# performance_demo()

## **Problem 1: Binary Words and Operations**

### 1.1 Parity Function
$\text{Parity}(x, y, z) = x \oplus y \oplus z$

#### 1.11 Design Decisions
**Old design decision**<br>
I chose to statically define all parameter types as NumPy 32-bit unsigned integers ``(np.uint32)`` to avoid writing defensive runtime checks and to improve performance.
In the Secure Hash Standard, this function would be called millions of times per hash computation. While this notebook is primarily meant to demonstrate functionality and could therefore benefit from more flexible, dynamically typed inputs I believe performance is just as critical as correctness in these bitwise operations. After all, these functions are designed to operate at the bit level for efficiency and precision; type checking them with expensive calls would defeat the purpose.

**Updated Design decision**<br>
- The core methods still define parameters with static typing for maximum performance.
- A new type-conversion decorator, ``@force_u32``, provides the best of both worlds; enabling automatic conversion of Python int values to ``np.uint32`` where appropriate,<br>
while leveraging existing code. This eliminates the need to maintain duplicate function definitions (one with type checks and one with static typing).
- All related methods are now encapsulated in static classes, giving the user explicit control over usage style:
    - ``ClassName.internal`` High-performance version (expects np.uint32 arguments).
    - ``ClassName.user_facing`` Decorated version with automatic type conversion for readability and convenience.

- Each static class also includes an ``assert_me`` method to streamline and simplify unit testing, improving clarity and reducing boilerplate.

This design allows you to call binary word operations directly with regular Python integers without manually wrapping every argument in ``u32()`` or ``np.uint32()``.<br>
At the same time, performance-critical code (such as within tight loops where functions can be called millions of times) can achieve up to 3× faster execution by using the undecorated internal method.

When calling the function from within another validated method, users may directly use the undecorated version, as the input types are already guaranteed to be ``np.uint32``.<br>
This avoids redundant type checks and maximizes performance.

In [ ]:
class Parity:
    # interal undecorated function
    @staticmethod
    def __Parity(x: u32, y: u32, z: u32) -> u32:
        """
        Operation:
            Applies bitwise eclusive or (XOR) to three 32-bit unsigned integers (x, y, z).
            For each bit position of x, y and z:
                - if an odd number of bits are 1, the result is 1
                - if an even number of bits are 1, the result is 0

        Parameters:
            (all arguments are of numpy unsigned 32-bit integer type)
            x: First integer
            y: Second integer.
            z: Third integer.

        Returns: 
            result of 'Operation' as numpy unsigned 32-bit integer.
        """

        # Apply bitwise XOR to the three integers and return the result
        return x ^ y ^ z
    
    # user-facing decorated function
    @staticmethod
    @force_u32
    def __Parity_safe(x: int, y: int, z: int) -> u32:
        """
        Wrapper function for Parity that converts all int inputs to np.uint32 safely.
        """
        return Parity.__Parity(x, y, z)
    
    # Unit test method template
    # improves readability and reduces boilerplate code in unit tests
    @staticmethod
    def assert_me(test_class: unittest.TestCase, x: int, y: int, z: int, 
               expected: int, fail_msg: str, print_equation: bool = False) -> None:
        """
        Overview:
            Asserts Parity function via TestCase.assertEqual() method.
                - used for readability in unit tests
                - can print detailed equation breakdown for debugging and verification.
                - automatically converts int inputs to np.uint32

        Parameters:
            test_class: The unittest.TestCase instance to use for assertions.
            x: First integer input for Parity.
            y: Second integer input for Parity.
            z: Third integer input for Parity.
            expected: The expected result of Parity(x, y, z).
            fail_msg: Message to display if the test fails.
            print_equation: If True, prints detailed equation breakdown.

        returns:
            None
        """
        
        # fetch result from Parity function
        result: u32 = Parity.__Parity_safe(x, y, z)

        # Optional detailed equation info printout
        if print_equation:
            print("----------------------------------------------------------------------")
            # print formatted test info
            print("Testing Parity with inputs: x:{} y:{} z:{}".format( x, y, z))
            print("Equation: {} ⊕ {} ⊕ {} = {} \n".format(x, y, z, result))
            print("Expected result:", expected)
            print("Actual result: {} \n".format(result))

            # detailed breakdown of the equation steps
            print("Equation breakdown:")
            # print inputs in binary format
            print("Inputs as binary: \n(x): {:032b} \
                  \n(y): {:032b} \n(z): {:032b} \n".format(x, y, z))
            
            # print step 1
            a_result: u32 = x ^ y
            print("Step 1: x ⊕ y:")
            print("(x): {:032b} \n(y): {:032b}".format(x, y))
            print("-------------------------- \n(a): {:032b} \n".format(a_result))

            # print step 2
            b_result: u32 = a_result ^ z
            print("Step 2: a ⊕ z:")
            print("(a): {:032b} \n(z): {:032b}".format(a_result, z))
            print("-------------------------- \nResult: {:032b} \n".format(b_result))

            # end section printout
            print("----------------------------------------------------------------------")

        # Assert that the result matches the expected value
        test_class.assertEqual(result, expected, fail_msg)

    # Expose methods to user
    # The preformance effect is negligible here, as they are just references
    # causing only 1 more dictionary lookup when called.
    internal = __Parity
    user_facing = __Parity_safe

### 1.11 Parity test cases

In [1414]:
# Unit tests for the Parity method.
class TestParity(unittest.TestCase):
    def test_base_cases(self):
        """Basic XOR combinations."""

        # 0 ⊕ 0 ⊕ 0 = 0
        Parity.assert_me(self, x=0, y=0, z=0, 
                        expected=0, 
                        fail_msg="Failed on all zeros")

        # 1 ⊕ 0 ⊕ 0 = 1
        Parity.assert_me(self, x=1, y=0, z=0, 
                        expected=1, 
                        fail_msg="Failed when only x=1")

        # 1 ⊕ 1 ⊕ 0 = 0
        Parity.assert_me(self, x=1, y=1, z=0,
                        expected=0,
                        fail_msg="Failed on even number of 1s")

        # 1 ⊕ 1 ⊕ 1 = 1
        Parity.assert_me(self, x=1, y=1, z=1, 
                        expected=1,
                        fail_msg="Failed on odd number of 1s")

    def test_non_trivial_cases(self):
        """Non-trivial test cases with larger integers."""

        # Test case 1
        Parity.assert_me(self, x=14, y=32, z=11,
                        expected=37,
                        fail_msg="Failed on non-trivial case 1",
                        print_equation=True)


run_local_scope_tests(TestParity, 2)

test_base_cases (__main__.TestParity.test_base_cases)
Basic XOR combinations. ... ok
test_non_trivial_cases (__main__.TestParity.test_non_trivial_cases)
Non-trivial test cases with larger integers. ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.002s

OK


----------------------------------------------------------------------
Testing Parity with inputs: x:14 y:32 z:11
Equation: 14 ⊕ 32 ⊕ 11 = 37 

Expected result: 37
Actual result: 37 

Equation breakdown:
Inputs as binary: 
(x): 00000000000000000000000000001110                   
(y): 00000000000000000000000000100000 
(z): 00000000000000000000000000001011 

Step 1: x ⊕ y:
(x): 00000000000000000000000000001110 
(y): 00000000000000000000000000100000
-------------------------- 
(a): 00000000000000000000000000101110 

Step 2: a ⊕ z:
(a): 00000000000000000000000000101110 
(z): 00000000000000000000000000001011
-------------------------- 
Result: 00000000000000000000000000100101 

----------------------------------------------------------------------


<unittest.runner.TextTestResult run=2 errors=0 failures=0>

### 1.2 Ch Function
$\text Ch(x, y, z) = (x \land y) \oplus (\lnot x \land z)$

In [1415]:
def Ch(x: u32, y:u32, z:u32) -> u32:
    return (x & y) ^ (~x & z)

In [1416]:
class TestCh(unittest.TestCase):

    def test_base_cases(self):
        """Basic Tests for Ch(x, y, z)."""

        # Case 1: x=0, y=0, z=0
        # (0 & 0) ⊕ (~0 & 0) = 0
        x1, y1, z1 = u32(0), u32(0), u32(0)
        expected1 = u32(0)
        self.assertEqual(Ch(x1, y1, z1), expected1, "Failed on all zeros")

        # Case 2: x=0, y=0, z=1
        # (0 & 0) ⊕ (~0 & 1) = 1
        x2, y2, z2 = u32(0), u32(0), u32(1)
        expected2 = u32(1)
        self.assertEqual(Ch(x2, y2, z2), expected2, "Failed when x=0, y=0, z=1")

        # Case 3: x=1, y=0, z=1
        # (1 & 0) ⊕ (~1 & 1) = 0
        x3, y3, z3 = u32(1), u32(0), u32(1)
        expected3 = u32(0)
        self.assertEqual(Ch(x3, y3, z3), expected3, "Failed when x=1, y=0, z=1")

        # Case 4: x=1, y=1, z=0
        # (1 & 1) ⊕ (~1 & 0) = 1
        x4, y4, z4 = u32(1), u32(1), u32(0)
        expected4 = u32(1)
        self.assertEqual(Ch(x4, y4, z4), expected4, "Failed when x=1, y=1, z=0")

        # Case 5: x=1, y=1, z=1
        # (1 & 1) ⊕ (~1 & 1) = 1
        x5, y5, z5 = u32(1), u32(1), u32(1)
        expected5 = u32(1)
        self.assertEqual(Ch(x5, y5, z5), expected5, "Failed when all are 1s")

        # Case 6: x=0, y=1, z=1
        # (0 & 1) ⊕ (~0 & 1) = 1
        x6, y6, z6 = u32(0), u32(1), u32(1)
        expected6 = u32(1)
        self.assertEqual(Ch(x6, y6, z6), expected6, "Failed when x=0, y=1, z=1")

# avoid redeclaring variables in global scope
run_local_scope_tests(TestCh)

----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

### 1.3 Maj Function
$\text Maj(x, y, z) = (x \land y) \oplus (x \land z) \oplus (y \land z)$

In [1417]:
def Maj(x: u32, y: u32, z: u32) -> u32:
    return (x & y) ^ (x & z) ^ (y & z)

#### 1.31 Maj Test Cases

In [1418]:
class TestMaj(unittest.TestCase):
    """Unit tests for the Maj(x, y, z) function."""

    def test_base_cases(self):
        """Base tests"""

        # Case 1: x=0, y=0, z=0
        # (0&0) ⊕ (0&0) ⊕ (0&0) = 0
        x1, y1, z1 = u32(0), u32(0), u32(0)
        expected1 = u32(0)
        self.assertEqual(Maj(x1, y1, z1), expected1, "Failed on all zeros")

        # Case 2: x=0, y=0, z=1
        # (0&0) ⊕ (0&1) ⊕ (0&1) = 0
        x2, y2, z2 = u32(0), u32(0), u32(1)
        expected2 = u32(0)
        self.assertEqual(Maj(x2, y2, z2), expected2, "Failed when z=1 only")

        # Case 3: x=0, y=1, z=1
        # (0&1) ⊕ (0&1) ⊕ (1&1) = 1
        x3, y3, z3 = u32(0), u32(1), u32(1)
        expected3 = u32(1)
        self.assertEqual(Maj(x3, y3, z3), expected3, "Failed when x=0, y=1, z=1")

        # Case 4: x=1, y=0, z=1 
        # (1&0) ⊕ (1&1) ⊕ (0&1) = 1
        x4, y4, z4 = u32(1), u32(0), u32(1)
        expected4 = u32(1)
        self.assertEqual(Maj(x4, y4, z4), expected4, "Failed when x=1, y=0, z=1")

        # Case 5: x=1, y=1, z=1
        # (1&1) ⊕ (1&1) ⊕ (1&1) = 1
        x5, y5, z5 = u32(1), u32(1), u32(1)
        expected5 = u32(1)
        self.assertEqual(Maj(x5, y5, z5), expected5, "Failed when all are 1s")

        # SHOW WORKINGS FOR 2 examples, basic and non-standard
        # Case 6: x=7, y=8, z=2
        0b00000111
        0b00001000
        # =
        0b00000000 # P1
        # XOR
        0b00000111
        0b00000010
        # =
        0b00000010 # P2
        # XOR
        0b00001000
        0b00000010
        # =
        0b00000000 # P3

        0b00000000 # P1
        0b00000010 # P2
        0b00000000 # P3
        0b00000010 # Final Result
        # 0b00000010 = 2


        # (7&8) ⊕ (7&2) ⊕ (8&2) = 2
        x6, y6, z6 = u32(7), u32(8), u32(2)
        expected6 = u32(2)
        self.assertEqual(Maj(x6, y6, z6), expected6, "Non-standard case")

# avoid redeclaring variables in global scope
run_local_scope_tests(TestMaj)

----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

### 1.4 Implementing Sigma functions
#### 1.41 Helper Functions
The sigma functions use the following methods throughout their implementation:
- **ROTR** (Circular right shift Operation)
- **SHR** ( Right Shift Operation )

These functions have been implemented as defined in [FIPS 180-4, Section 3.2: Operations on Words](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf) as helper methods to reduce repetiton. 

**Circular right shift Operation**<br>
$\text{ROTR}^n(x) = (x \gg n) \lor (x \ll w - n)$

In [1419]:
def ROTR(x: u32, n: u32) -> u32:
    """Right rotate a 32-bit word x by n bits."""
    return (x >> n) | (x << (32 - n))

**Right Shift Operation**<br>
$\text{SHR}^n(x) = x \gg n$

In [1420]:
def SHR(x: u32, n:u32) -> u32:
    """Logical right shift of x by n bits."""
    return (x >> n)

#### 1.5 large Sigma 0
$\Sigma_{0}^{256}(x) \;=\; \mathrm{ROTR}^2(x) \;\oplus\; \mathrm{ROTR}^{13}(x) \;\oplus\; \mathrm{ROTR}^{22}(x)$

In [1421]:
def Sigma0(x: u32) -> u32:
    return ROTR(x, u32(2)) ^ ROTR(x, u32(13)) ^ ROTR(x, u32(22))

#### 1.6 Large Sigma 1
$\Sigma_{1}^{256}(x) \;=\; \mathrm{ROTR}^6(x) \;\oplus\; \mathrm{ROTR}^{11}(x) \;\oplus\; \mathrm{ROTR}^{25}(x)$

In [1422]:
def Sigma1(x: u32) -> u32:
    return ROTR(x, u32(6)) ^ ROTR(x, u32(11)) ^ ROTR(x, u32(25))

#### 1.7 Small Sigma 0
$\sigma_{0}^{256}(x) = \mathrm{ROTR}^{7}(x) \;\oplus\; \mathrm{ROTR}^{18}(x) \;\oplus\; \mathrm{SHR}^{3}(x)$

In [1423]:
def sigma0(x: u32) -> u32:
    return ROTR(x, u32(7)) ^ ROTR(x, u32(18)) ^ SHR(x, u32(3))

#### 1.7 Small Sigma 1
$\sigma_{1}^{256}(x) = \mathrm{ROTR}^{17}(x) \;\oplus\; \mathrm{ROTR}^{19}(x) \;\oplus\; \mathrm{SHR}^{10}(x)$

In [1424]:
def sigma1(x: u32) -> u32:
    return ROTR(x, u32(17)) ^ ROTR(x, u32(19)) ^ SHR(x, u32(10))

## Problem 2: Fractional Parts of Cube Roots

## Problem 3: Padding

## Problem 4: Hashes

## Problem 5: Passwords